# This code will serve as a way to organize the corpus before training the Seq2seq model

#### Previous version : separation by parts (VERSE/INTRO/ETC...)
#### New version : separation by parts / by 2 lines 

What this version will need for context of the Seq2seq :

- The length of the parts

- The previous two lines of txt, if no lines specific embedding

- The parts name

- At what stage appear our text : if appear at the beginning ==> 0/length_part = 0 / at the middle maybe 0.5 at the end 1

if we are at the beginning and at the end, (less than two lines in a part), I will choose to set it to 1

In [5]:
import re
import pickle
import pandas as pd
import numpy as np
import itertools
import os

In [6]:
list_files = os.listdir("../Corpus/Cleaning/")

new_corp = []
for i in list_files :
    if "clean_corpus" in i  :
        with open(f"../Corpus/Cleaning/{i}", "r", encoding="utf-8", errors="replace") as f:
            corpus = f.read()
            new_corp.extend(corpus)
corpus = "".join(new_corp)
len(corpus)

991587

ok now the corpus will be modified to be able to be used in a Seq2seq

In [180]:
corpus = corpus.replace("§","<EOS>")
corpus = corpus.replace("\n","<EOL>")

#### Pre-select the songs with parts

In [181]:
all_inside_parts = re.findall(r"<BEGINNING><EOL>(.*?)<END>", corpus, flags=re.DOTALL)

new_inside = []
for j in all_inside_parts :
    if j[0] == "<" :
        new_inside.append(j)

print(len(all_inside_parts),len(new_inside))

443 417


#### Get all the parts indication

In [182]:
all_parts_indic = []
for i in new_inside :
    parts_indic = re.findall(r"<(\w+>)<EOL>.*?<EOL><EOS>", i, flags=re.DOTALL)
    for i in range(len(parts_indic)) :
        parts_indic[i] = "<PART="+parts_indic[i]

    all_parts_indic.extend(parts_indic)
    #For previous version use extend

len(all_parts_indic)

1444

In [183]:
all_parts_indic[0]

'<PART=REFRAIN>'

#### Get all the inside part

In [184]:
all_inside_parts = []
for i in new_inside :
    all_inside = re.findall(r"<\w+><EOL>(.*?)<EOL><EOS>", i, flags=re.DOTALL)
    all_inside_parts.extend(all_inside)

In [185]:
all_inside_parts[1]

"On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément<EOL>On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément"

#### Next regroup the lines inside a part by group of 2

In [186]:
all_inside_parts_sep = []
all_inside_parts_indic = []

for part in all_inside_parts :
    new = []
    other_new = []

    inside = part.split("<EOL>")
    for j in range(len(inside)) :
        if j%2!=0 : #Only if odd number, indicate that we can have a tuple

        #We can also use the same code to add the two previous lines to the context
            if j==1 : #First two lines have no previous lines in the part
                other_new.append(["<START>","<START>"])
            elif j>1 :
                other_new.append(new[-1])
            new.append([inside[j-1],inside[j]])

        #Pb if the last j isn't odd then the last lines isn't added into it
        elif (j == len(inside)-1) and (j%2==0) :
            if j==0 : #Only one line so no previous line to append
                other_new.append(["<START>","<START>"])
            else : 
                other_new.append(new[-1])
            new.append([inside[j],'<NOTHING>'])

    all_inside_parts_sep.append(new)
    all_inside_parts_indic.append(other_new)

In [187]:
len(all_inside_parts_indic[1]),len(all_inside_parts_sep[1])

(2, 2)

#### Get the nb of group of lines per parts

In [188]:
nb_lines = []

for j in all_inside_parts :
    nb = j.count("<EOL>")+1
    if nb%2==0 :
        adapt_nb = nb/2
    else :
        adapt_nb = (nb+1)/2
    nb_lines.append(int(adapt_nb))

#### Get the progression 

In [189]:
mapping_inside_lines = []
inside_lines = []

for i in nb_lines:
    inside_ = []
    if i%2!= 0 :
        for j in range(i) :
            inside_.append(j+1)
            if j not in mapping_inside_lines :
                mapping_inside_lines.append(j+1)
    else : 
        for j in range(i) :
            inside_.append(j+1)
            if j not in mapping_inside_lines :
                 mapping_inside_lines.append(j+1)
    inside_lines.append(inside_)

Now little check to see if everything works as intended

In [190]:
print(all_parts_indic[1])
print(all_inside_parts[1])
for j in range(len(all_inside_parts_sep[1])) :
    print(inside_lines[1][j],nb_lines[1],"\nPrevious_lines :",all_inside_parts_indic[1][j], "\nActual lines :",all_inside_parts_sep[1][j])

<PART=REFRAIN>
On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément<EOL>On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément
1 2 
Previous_lines : ['<START>', '<START>'] 
Actual lines : ["On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh", "J'suis dans mon élément eh, j'suis dans mon élément"]
2 2 
Previous_lines : ["On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh", "J'suis dans mon élément eh, j'suis dans mon élément"] 
Actual lines : ["On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh", "J'suis dans mon élément eh, j'suis dans mon élément"]


### Ok everything for the context is done, now I need to regroup them into a context variable

In [191]:
for i in range(len(all_parts_indic)): 
    print(all_parts_indic[i])
    print(all_inside_parts[i])
    for j in range(len(all_inside_parts_sep[i])) :
        print(inside_lines[i][j],nb_lines[i],"\nPrevious_lines :",all_inside_parts_indic[i][j], "\nActual lines :",all_inside_parts_sep[i][j])

    break

<PART=REFRAIN>
On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément<EOL>On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément<EOL>On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément<EOL>On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh<EOL>J'suis dans mon élément eh, j'suis dans mon élément
1 4 
Previous_lines : ['<START>', '<START>'] 
Actual lines : ["On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j'suis dans mon élément, eh", "J'suis dans mon élément eh, j'suis dans mon élément"]
2 4 
Previous_lines : ["On vit sous tension et c'est tout l'temps sombre et j'm'en fous dans l'fond, j's

In [192]:
context = []
context_two = []

for i in range(len(all_parts_indic)): 
    for j in range(len(all_inside_parts_sep[i])) :
#I need to separate the two because Previous will be a list of list inside a list and that will not be able to save to parquet easily
        context.append([all_parts_indic[i],"<lines="+str(inside_lines[i][j])+">","<total="+str(nb_lines[i])+">"])
        context_two.append(["Previous :",all_inside_parts_indic[i][j][0],"Previous :",all_inside_parts_indic[i][j][1]])

In [193]:
set(all_parts_indic)

{'<PART=COUPLET>',
 '<PART=INTRO>',
 '<PART=OUTRO>',
 '<PART=PONT>',
 '<PART=REFRAIN>'}

#### Now that I have everything, I can prepare for the new vocab mapping

In [194]:
mapping_inside_lines = []
for i in inside_lines :
    for j in i :
        if j not in mapping_inside_lines: 
            mapping_inside_lines.append("<lines="+str(j)+">")
set(mapping_inside_lines)

{'<lines=10>',
 '<lines=11>',
 '<lines=12>',
 '<lines=13>',
 '<lines=14>',
 '<lines=15>',
 '<lines=16>',
 '<lines=17>',
 '<lines=18>',
 '<lines=19>',
 '<lines=1>',
 '<lines=20>',
 '<lines=21>',
 '<lines=22>',
 '<lines=23>',
 '<lines=24>',
 '<lines=25>',
 '<lines=26>',
 '<lines=27>',
 '<lines=28>',
 '<lines=29>',
 '<lines=2>',
 '<lines=30>',
 '<lines=31>',
 '<lines=32>',
 '<lines=33>',
 '<lines=34>',
 '<lines=35>',
 '<lines=36>',
 '<lines=37>',
 '<lines=38>',
 '<lines=39>',
 '<lines=3>',
 '<lines=40>',
 '<lines=41>',
 '<lines=42>',
 '<lines=43>',
 '<lines=44>',
 '<lines=45>',
 '<lines=46>',
 '<lines=47>',
 '<lines=48>',
 '<lines=49>',
 '<lines=4>',
 '<lines=50>',
 '<lines=51>',
 '<lines=52>',
 '<lines=53>',
 '<lines=54>',
 '<lines=55>',
 '<lines=56>',
 '<lines=57>',
 '<lines=58>',
 '<lines=59>',
 '<lines=5>',
 '<lines=60>',
 '<lines=61>',
 '<lines=62>',
 '<lines=63>',
 '<lines=6>',
 '<lines=7>',
 '<lines=8>',
 '<lines=9>'}

In [195]:
mapping_nb_lines = []
for i in set(nb_lines) :
    mapping_nb_lines.append("<total="+str(i)+">")

In [196]:
list(set(all_parts_indic))

['<PART=INTRO>',
 '<PART=OUTRO>',
 '<PART=PONT>',
 '<PART=REFRAIN>',
 '<PART=COUPLET>']

Creation of mapping

In [197]:
full_vocab = []
new_all_inside_parts = []
for songs in all_inside_parts_sep :

    new_parts = []
    for parts in songs :
        
        new_two_lines = []
        for two_lines in parts :
            if len(new_two_lines) == 0 :
                new_two_lines.extend(two_lines)
                new_two_lines.append('<EOL>')
            else : 
                new_two_lines.extend(two_lines)
        full_vocab.extend(new_two_lines)

        new_parts.append(new_two_lines)
    new_all_inside_parts.append(new_parts)


In [198]:
special = list(set(all_parts_indic))+ list(set(mapping_nb_lines))+list(set(mapping_inside_lines))
special

['<PART=INTRO>',
 '<PART=OUTRO>',
 '<PART=PONT>',
 '<PART=REFRAIN>',
 '<PART=COUPLET>',
 '<total=3>',
 '<total=21>',
 '<total=41>',
 '<total=20>',
 '<total=22>',
 '<total=6>',
 '<total=63>',
 '<total=9>',
 '<total=27>',
 '<total=16>',
 '<total=23>',
 '<total=11>',
 '<total=2>',
 '<total=7>',
 '<total=17>',
 '<total=18>',
 '<total=28>',
 '<total=5>',
 '<total=19>',
 '<total=13>',
 '<total=36>',
 '<total=44>',
 '<total=29>',
 '<total=12>',
 '<total=25>',
 '<total=35>',
 '<total=39>',
 '<total=30>',
 '<total=8>',
 '<total=14>',
 '<total=1>',
 '<total=10>',
 '<total=24>',
 '<total=15>',
 '<total=4>',
 '<lines=19>',
 '<lines=43>',
 '<lines=61>',
 '<lines=16>',
 '<lines=54>',
 '<lines=55>',
 '<lines=37>',
 '<lines=23>',
 '<lines=5>',
 '<lines=22>',
 '<lines=47>',
 '<lines=60>',
 '<lines=26>',
 '<lines=18>',
 '<lines=14>',
 '<lines=40>',
 '<lines=4>',
 '<lines=50>',
 '<lines=21>',
 '<lines=56>',
 '<lines=30>',
 '<lines=44>',
 '<lines=49>',
 '<lines=51>',
 '<lines=15>',
 '<lines=35>',
 '<lines

Two vocabs : One for the Encodeur and One for the Decodeur

In [199]:
encode_vocab = {ch: i for i, ch in enumerate(["<PAD>"]+list(set(full_vocab))+special+["<START>"]+["Previous :"])}

In [200]:
decode_vocab = {ch: i for i, ch in enumerate(["<PAD>"]+list(set(full_vocab))+["START","END"])}

In [201]:
global_vocab = {ch: i for i, ch in enumerate(["<PAD>"]+sorted(list(set(full_vocab)))+special+["<START>"]+["Previous :"]+["START"]+["END"])}

#### Now we need to first convert our datas to values

In [202]:
flat_input = list(itertools.chain.from_iterable(new_all_inside_parts))

We need to add the first token indicating the start and the final token

In [203]:
final_flat = []

for i in flat_input :
    inter_ = ["START"]
    inter_.extend(i)
    inter_ +=["END"]
    final_flat.append(inter_)

## Pre-tokenize everything

In [204]:
context_encode = [[global_vocab[i] for i in elem] for elem in context]
final_encoded = [[global_vocab[i] for i in elem] for elem in final_flat]
#context_two_encode = [[char2int[i] if (i) in ("Previous :","<START>") else [char2int[j] for j in i]  for i in elem] for elem in context_two]

In [205]:
context_two_encode = []
for i in (context_two) :
    inter_contest_two = []
    for j in i :
        if "Previous" in j or "<START>" in j :
            inter_contest_two.append(global_vocab[j])
        else : 
            inter_contest_two.extend([global_vocab[char] for char in j])
    context_two_encode.append(inter_contest_two)

In [206]:
context_final = [a + b for a, b in zip(context_encode, context_two_encode)]

## Modify the dataset to have a faster training and less compute

In [207]:
final_encoded_target = [elem[1:] for elem in final_encoded]
final_encoded_input = [elem[:-1] for elem in final_encoded]

I can save directly into np.arrays to save a bit of time

In [208]:
lengths = np.array([len(s) for s in context_final], dtype=np.int16)
offsets = np.zeros(len(context_final), dtype=np.int32)
offsets[1:] = np.cumsum(lengths)[:-1]
context_np = np.array(list(itertools.chain.from_iterable(context_final)),dtype=np.int16)

In [209]:
np.save("Files/files_npy/context_np.npy",context_np)
np.save("Files/files_npy/context_length.npy",lengths)
np.save("Files/files_npy/context_offset.npy",offsets)

In [210]:
lengths = np.array([len(s) for s in final_encoded_target], dtype=np.int16)
offsets = np.zeros(len(final_encoded_target), dtype=np.int32)
offsets[1:] = np.cumsum(lengths)[:-1]
final_np = np.array(list(itertools.chain.from_iterable(final_encoded_target)),dtype=np.int16)

In [211]:
np.save("Files/files_npy/Y_np.npy",final_np)
np.save("Files/files_npy/Y_length.npy",lengths)
np.save("Files/files_npy/Y_offset.npy",offsets)

In [212]:
lengths = np.array([len(s) for s in final_encoded_input], dtype=np.int16)
offsets = np.zeros(len(final_encoded_input), dtype=np.int32)
offsets[1:] = np.cumsum(lengths)[:-1]
final_np = np.array(list(itertools.chain.from_iterable(final_encoded_input)),dtype=np.int16)

In [213]:
np.save("Files/files_npy/X_np.npy",final_np)
np.save("Files/files_npy/X_length.npy",lengths)
np.save("Files/files_npy/X_offset.npy",offsets)

# Create the dataset that will contain everything

In [214]:
with open("Files/Encoding_map.pkl", "wb") as f:
    pickle.dump(global_vocab, f)